# Gestion du cache

Le principe d'un cache est de conserver les résultats de requêtes pour ne pas avoir à les recalculer lorsque ces requêtes seront à nouveau reçues. Le cache est intéressant à partir du moment où des requêtes reviennent assez fréquemment, sachant qu'en contrepartie, la cache va consommer de la ressource mémoire, disque et CPU. Il faut que cette consommation de ressources soit plus rentable que la consommation faite par le calcul qu'implique la requête reçue, pour que le cache soit considéré comme une option intéressante.

Spark propose une fonctionnalité de cache au niveau Spark Core sur les RDD et Spark SQL sur les dataframe et les dataset. Dans le cadre de Spark SQL, le cache devient intéressant dès qu'un dataframe est utilisé dans plusieurs requêtes séparées. Dans cette situation, l'utilisation du cache pour un dataframe permet au niveau de chacune des requêtes qui utilisent ce dataframe de réévaluer à chaque fois les calculs qui ont conduit à ce dataframe.

## Prélude

In [ ]:
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`
import $ivy.`org.apache.logging.log4j:log4j-api:2.8.2`
import $ivy.`org.apache.logging.log4j:log4j-slf4j-impl:2.8.2`

// Avoid disturbing logs
import org.apache.log4j._
import org.apache.log4j.varia._
BasicConfigurator.configure(NullAppender.getNullAppender())

import $ivy.`org.apache.spark::spark-core:3.4.1`
import $ivy.`org.apache.spark::spark-sql:3.4.1`

In [ ]:
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.storage.StorageLevel

val spark = {
  NotebookSparkSession.builder()
    .progress(enable = true, keep = true, useBars = false)
    .master("local[*]")
    .appName("Spark tuning — Cache")
    .getOrCreate()
}

import spark.implicits._
val sparkContext = spark.sparkContext
import $file.^.internal.spark_helper
import spark_helper.implicits._

In [ ]:
def callWithName[A](name: String)(f: => A): A = {
  spark.sparkContext.setCallSite(name)
  try f
  finally spark.sparkContext.clearCallSite()
}

## Introduction sur le cache

In [ ]:
%%data limit=10

val orders: DataFrame =
  spark.read
    .option("header", true)
    .csv("data/orders.csv")
    .repartition(4)

orders.printSchema()
orders

### Sans cache

Nous voulons effectuer des analyses sur la journée du 1er février 2023, sans utiliser de cache.

In [ ]:
%%data limit=10

val partOfOrders = orders.where(to_date($"timestamp") === to_date(lit("2023-02-01")))

partOfOrders

Nous récupérons d'abord le chiffre d'affaires de la journée.

In [ ]:
callWithName("show revenue - no cache") {
  partOfOrders
    .agg(round(sum($"price"), 2) as "total")
    .show()
}

Nous allons ensuite analyser la répartition du chiffre d'affaires de la journée sur les différents produits.

In [ ]:
callWithName("show product revenue - no cache") {
  partOfOrders
    .groupBy($"product")
    .agg(sum($"price") as "total")
    .where($"total" >= 15.0)
    .orderBy($"total".desc)
    .show()
}

Nous récupérons enfin la répartition des ventes par client sur le café noisette et le total des ventes de la journée sur ce produit.

In [ ]:
callWithName("show noisette - no cache") {
  partOfOrders
    .where($"product" === "noisette")
    .cube($"client")
    .agg(sum($"price") as "total")
    .orderBy($"client".desc)
    .show()
}

👀 **Question** 👀

Dans toutes ces requêtes, lorsque nous regardons le plan d'exécution, nous voyons des DAG relativement équivalents. De plus chaque requête se traduit par l'exécution de 3 jobs. Nous le voyons dans ce notebook, au niveau des barres de progression.

### Avec cache

Nous allons maintenant exécuter les mêmes requêtes. Mais cette fois, nous allons mettre en cache le dataframe filtré sur la journée 1er février 2023.

In [ ]:
%%data limit=10

val partOfOrders = orders.where(to_date($"timestamp") === to_date(lit("2023-02-01"))).cache()

partOfOrders

👷 Exécutez les requêtes ci-dessous.

In [ ]:
callWithName("show revenue - cache") {
  partOfOrders
    .agg(round(sum($"price"), 2) as "total")
    .show()
}

In [ ]:
callWithName("show product revenue - cache") {
  partOfOrders
    .groupBy($"product")
    .agg(sum($"price") as "total")
    .where($"total" >= 15.0)
    .orderBy($"total".desc)
    .show()
}

In [ ]:
callWithName("show noisette - cache") {
  partOfOrders
    .where($"product" === "noisette")
    .cube($"client")
    .agg(sum($"price") as "total")
    .orderBy($"client".desc)
    .show()
}

👀 **Ce qu'il faut voir** 👀

La différence, qui apparaît ici, est que l'exécution des requêtes d'analyse nécessite cette 2 jobs au lieu de 3. Ce qui permet un gain en performance sur l'exécution de ces requêtes.

En regardant de plus près le plan d'exécution de ces requêtes, nous remarquons que celui-ci est un plus grand. Dans le cadre de la mise en cache, Spark a ajouté les étapes :

* **InMemoryRelation** : mise en cache des données du dataframe
* **InMemoryTableScan** : parcours des données du dataframe depuis le cache

Ces étapes proviennent de `partOfOrders`.

=> 👷 Exécutez la cellule suivante pour voir le plan d'exécution lié à `partOfOrders`.

In [ ]:
partOfOrders.explain()

En fait, dans les requêtes d'analyse, toutes les étapes précédentes _InMemoryTableScan_ ne sont pas exécutées.

Si vous allez dans Spark UI, dans l'onglet _Storage_ vous allez trouver un tableau avec une seule entrée, contenant les informations ci-dessous :

* Storage Level: Disk Memory Deserialized 1x Replicated
* Cached Partitions: 4
* Fraction Cached: 100%
* Size in Memory: 8.6 KiB
* Size on Disk: 0.0 B

Cette ligne unique indique les informations sur les données en cache liées à `partOfOrders`.

_Storage Level_ indique le type de stockage utilisé. Ici, nous avons un stockage en mémoire désérialisé (il s'agit uniquement d'objets Java stockés dans la _heap_ de la JVM) et sur disque, avec un unique répliqua (il n'y a pas de réplication sur d'autres executor).

Nous avons aussi la quantité de mémoire utilisée et la quantité de données stockée sur disque.

Si vous cliquez sur la ligne du tableau dans Spark UI, vous obtenez plus d'informations sur nos données en cache. Nous avons notamment la répartition des données en cache par partition avec les executor associés. Et nous avons aussi la mémoire restante pour le cache.

Il est possible de supprimer le cache à la main en utilisant la méthode `.unpersist()`.

In [ ]:
partOfOrders.unpersist()

Si vous retournez dans l'onglet _Storage_ de Spark UI, vous verrez que le tableau aura disparu.

### Fonctionnement

Le cache Spark SQL est un cache LRU (_Least Recently Used_). Ainsi, lorsque l'espace pour le cache manque, toutes les entrées les moins utilisées sont retirées du cache pour laisser de la place. Lors du prochain accès aux données retirées, Spark recalculera ces données pour les réintégrer dans le cache, en ayant au préalable retiré les données les moins utilisées, si le cache est à nouveau plein.

## Les différents niveaux de cache

### CacheManager

Spark SQL bénéficie d'un CacheManager qui centralise la gestion des caches.

Il est possible de vider complètement le cache à travers son interface.

In [ ]:
val cacheManager = spark.sharedState.cacheManager

cacheManager.clearCache()

Voyons le fonctionnement du CacheManager. Pour cela, nous allons mettre en cache 2 requêtes identiques, mais réclamant une stratégie du stockage du cache différente.

In [ ]:
val orders_default: DataFrame =
  spark.read.option("header", true).csv("data/orders.csv").repartition(4).cache()
orders_default.take(1)

In [ ]:
val orders_memory: DataFrame =
  spark.read.option("header", true).csv("data/orders.csv").repartition(4).persist(StorageLevel.MEMORY_ONLY)
orders_default.take(1)

👀 **Ce qu'il faut voir** 👀

Si nous regardons dans Spark UI, bien que nous ayons produits deux dataframe, nous ne verrons qu'un seul cache dans l'onglet _Storage_.

Plus exactement, la recherche du cache se fait selon l'opération suivante :

In [ ]:
orders_default.queryExecution.logical
  .sameResult(
    orders_memory.queryExecution.logical
  )

Ce qui veut dire que ce qui est retenu, c'est plus exactement la similarité des résultats obtenus par les dataframes et non pas sur l'égalité entre les plans d'exécution des dataframes.

La difficulté est que tout ceci est déterminé avant l'exécution de la requête. Il est dans ce cas parfois difficile de déterminer si deux plans d'exécution produiront des résultats équivalents. Spark ne sait pas toujours le déterminer. Par contre, Spark va capable de déterminer si deux plans d'exécution vont donner des résultats différents.

Pour retrouver des données en cache, Spark SQL passe par la méthode `.lookupCachedData()` du CacheManager.

In [ ]:
object test_unique_cache {
  val cache_orders_default = cacheManager.lookupCachedData(orders_default).get
  val cache_orders_memory = cacheManager.lookupCachedData(orders_memory).get

  def run() = {
    println("cache_orders_default.plan sameResult cache_orders_memory.plan: " + cache_orders_default.plan.sameResult(cache_orders_memory.plan))
    println("cache_orders_default: " + cache_orders_default.cachedRepresentation.cacheBuilder.storageLevel)
    println("cache_orders_memory: " + cache_orders_memory.cachedRepresentation.cacheBuilder.storageLevel)
  }
}

test_unique_cache.run()

D'ailleurs, notre dataframe d'origine est lui aussi indexé dans le cache, même si son plan d'exécution déduit n'a pas changé.

In [ ]:
orders.explain()
cacheManager.lookupCachedData(orders).isDefined

Si vous exécutez la cellule ci-dessous et que vous allez dans Spark UI, vous verrez apparaître une étape InMemoryRelation et InMemoryTableScan, indiquant que les données pour cette requête n'ont pas été récupérées depuis le fichier, mais depuis le cache.

In [ ]:
callWithName("take from order - before clear cache") {
  orders.take(1)
}

Par contre, si nous vidons le cache, la récupération des données se fait depuis le fichier directement. Dans le plan d'exécution, dans Spark UI, les étapes InMemoryRelation et InMemoryTableScan auront disparues.

In [ ]:
cacheManager.clearCache()

orders.explain()
cacheManager.lookupCachedData(orders).isDefined

callWithName("take from order - after clear cache") {
  orders.take(1)
}

### Stockage des caches

Voici la liste des stratégies de stockage proposée par Spark.

* **NONE** : pas de persistance (empêche le CacheManager de persister par la suite un type de requête)
* **DISK_ONLY** : persistance sur le disque local
* **DISK_ONLY_2** : persistance sur disque répliquée sur 2 nœuds
* **DISK_ONLY_3** : persistance sur disque répliquée sur 3 nœuds
* **MEMORY_ONLY** : persistance en mémoire
* **MEMORY_ONLY_2** : persistance en mémoire sur 2 nœuds
* **MEMORY_ONLY_SER** : persistance sérialisée en mémoire
* **MEMORY_ONLY_SER_2** : persistance sérialisée en mémoire sur 2 nœuds
* **MEMORY_AND_DISK** : persistance en mémoire et sur disque
* **MEMORY_AND_DISK_2** : persistance en mémoire et sur disque répliquée sur 2 nœuds
* **MEMORY_AND_DISK_SER** : persistance sérialisée en mémoire et sur disque
* **MEMORY_AND_DISK_SER_2** : persistance sérialisée en mémoire et sur disque répliquée sur 2 nœuds
* **OFF_HEAP** : persistance en mémoire _off heap_ (_expériemntal_)

La sérialisation proposée ici utilise un encodage binaire propre à Spark et qui reconnu comme efficace. Il permet de gagner de l'espace en mémoire, en demandant un effort supplémentaire sur le CPU.

Le stockage sur disque est bien évidemment plus coûteux que le stockage en mémoire. Il faut dans ce cas que le recalcul du dataframe soit plus lent que l'accès disque pour récupérer les données.

La réplication est intéressante si le recalcul du dataframe est extrêmement lent et que l'on souhaite donner la possibilité à Spark de reprendre les traitements en cours sur un autre executor, en cas de panne partielle.

**Exercice**

👷 Ci-dessous, après avoir vidé le cache, vérifier dans Spark UI l'effet du stockage `NONE`.

In [ ]:
cacheManager.clearCache()

In [ ]:
callWithName("StorageLevel.NONE") {
  orders.where(to_date($"timestamp") === to_date(lit("2023-02-28"))).persist(StorageLevel.NONE).take(1)
}

👷 Exécutez la cellule ci-dessous.

In [ ]:
callWithName("StorageLevel.MEMORY_ONLY") {
  val memory_only =
    orders.where(to_date($"timestamp") === to_date(lit("2023-03-01"))).persist(StorageLevel.MEMORY_ONLY).take(1).toList
  println(s"memory_only = $memory_only")
}
callWithName("StorageLevel.MEMORY_ONLY_SER") {
  val memory_only_ser =
    orders.where(to_date($"timestamp") === to_date(lit("2023-03-02"))).persist(StorageLevel.MEMORY_ONLY_SER).take(1).toList
  println(s"memory_only_ser = $memory_only_ser")
}
callWithName("StorageLevel.DISK_ONLY") {
  val disk_only =
    orders.where(to_date($"timestamp") === to_date(lit("2023-03-03"))).persist(StorageLevel.DISK_ONLY).take(1).toList
  println(s"disk_only = $disk_only")
}
callWithName("StorageLevel.MEMORY_AND_DISK") {
  val memory_and_disk =
    orders.where(to_date($"timestamp") === to_date(lit("2023-03-04"))).persist(StorageLevel.MEMORY_AND_DISK).take(1).toList
  println(s"memory_and_disk = $memory_and_disk")
}
callWithName("StorageLevel.MEMORY_AND_DISK_SER") {
  val memory_and_disk_ser =
    orders.where(to_date($"timestamp") === to_date(lit("2023-03-05"))).persist(StorageLevel.MEMORY_AND_DISK_SER).take(1).toList
  println(s"memory_and_disk_ser = $memory_and_disk_ser")
}
// experimental
callWithName("StorageLevel.OFF_HEAP") {
  val off_heap =
    orders.where(to_date($"timestamp") === to_date(lit("2023-03-06"))).persist(StorageLevel.OFF_HEAP).take(1).toList
  println(s"off_heap = $off_heap")
}

🤔 **Question** 🤔

Vérifiez dans Spark UI la stratégie de stockage qui semble la meilleure. Appuyez-vous pour cela sur les statistiques de latence et de consommation d'espace.

----

Nous allons maintenant voir l'effet d'une stratégie de stockage impliquant une réplication, sachant que nous n'avons qu'un seul executor (ie. le driver).

Après avoir vidé le cache et lancé la requête, regardez dans Spark UI, dans l'onglet _Storage_, si un espace de stockage a été créé.

In [ ]:
cacheManager.clearCache()
callWithName("StorageLevel.DISK_ONLY_2") {
  orders.where(to_date($"timestamp") === to_date(lit("2023-03-07"))).persist(StorageLevel.DISK_ONLY_2).take(1)
}